In [1]:
import math
import random

In [ ]:
def getRandomBoard(dimension):
    """Retorna um array que representa o tabuleiro (dimension x dimension) no qual cada coluna tem uma rainha posicionada"""
    